In [ ]:
# Script to cluster GAEZ data using Agglomerative Heirarichal clustering method

import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import seaborn as sns; sns.set(color_codes=True)
import matplotlib
import numpy as np
import itertools
import scipy.cluster.hierarchy as hac

from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster

from tkinter import filedialog
from tkinter import *

%matplotlib auto

print('Python version ' + sys.version)
print('Matplotlib version ' + matplotlib.__version__)
print('Pandas version ' + pd.__version__)


In [ ]:
# Prompt user to select folder that contains required data files
root = Tk()
root.folder =  filedialog.askdirectory()
root.destroy()

In [ ]:
# Set user-defined parameters 

#Ethiopia
#regions = {1: "ETH"
#          }

## Viet Nam
#regions = {1: "CEC", #Central_45Coco
#           2: "CES", #Central_69SanJuan
#           3: "ATM", #Atlantico_55Mat
#           4: "ATE", #Atlantico_61Esc
#           5: "PAC", #Pacifico
#           6: "SWE", #South-west of 'Rest of Nicaragua'
#           7: "SEA", #South-east of 'Rest of Nicaragua'
#           8: "NEA", #North-east of 'Rest of Nicaragua'
#          } # Regions to be clustered

## Indonesia

#regions = {1:"IND"}
regions = {1: "JWA",
           2: "KAN",
           3: "MLU",
           4: "NUA",
           5: "PPA",
           6: "SLI",
           7: "SMA"} # Regions to be clustered

#regions = {1: "ADA",
#           2: "CEN",
#           3: "EAS",
#           4: "EXN",
#           5: "LIT",
#           6: "NWE",
#           7: "NOR",
#           8: "WES",
#           9: "SWE", 
#           10: "SOU"}

#Ethiopia
#k = {1:1
#    }

## Viet Nam
#k = {1: 1,
#     2: 1,
#     3: 1,
#     4: 1,
#     5: 1,
#     6: 1,
#     7: 1,
#     8: 1,
#    } # Selected number of clusters, using 0.2 threshold in elbow graphs


## Indonesia
#k = {1:45}
k = {1: 2,
     2: 3,
     3: 2,
     4: 2,
     5: 2,
     6: 2,
     7: 3} # Selected number of clusters, using 0.2 threshold in elbow graphs

## Crop list for Viet Nam
#crop_list = {'Barley': ['Barley'],
#             'Maize': ['Maize'],
#             'Sorghum': ['Sorghum'],
#             'Sugarcane': ['Sugarcane'],
#             'Sweet Potato': ['Sweet Potato'],
#             'Teff': ['Warm C4'],
#             'Wheat': ['Wheat'],
#             'Others': ['Cassava'],
#             #'Rubber': ['Jatropha'],
#             #'Vegetables and fruits': ['Cabbage','Carrot','Onion','Tomato','Banana','Citrus','Coconut','Cocoa'],
#             #'Sweet potato': ['Sweet potato']
#            }

## Crop list for Viet Nam
#crop_list = {'Maize': ['Maize'],
#             'Beans': ['Phaseolus bean'],
#             'Coffee': ['Coffee'],
#             'Sugarcane': ['Sugarcane'],
#             'Rice': ['Wetland rice'],
#             'Sorghum': ['Sorghum'],
#             'Groundnuts': ['Groundnut'],
#             'Cocoa': ['Cocoa'],
#             'Others': ['Cassava'],
#             #'Rubber': ['Jatropha'],
#             #'Vegetables and fruits': ['Cabbage','Carrot','Onion','Tomato','Banana','Citrus','Coconut','Cocoa'],
#             #'Sweet potato': ['Sweet potato']
#            }

## Crop list for Indonesia
crop_list = {'Wetland rice': ['Wetland rice'],
             #'Fruits and Vegetables': ['Cabbage','Carrot','Onion','Tomato', 'Banana','Citrus','Cocoa'],
             'Palm oil': ['Oilpalm'],
             'Coconut': ['Coconut'],
             'Rubber': ['Jatropha'],
             #'Coffee': ['Coffee'],
             #'Tea': ['Tea'],
             #'Tobacco': ['Tobacco'],
             'Maize': ['Maize'],
             'Other': ['White potato', 'Cassava', 'Cocoa', 'Coffee', 'Onion', 'Soybean', 'Sugarcane'],
             }

#land_area_cell = 0.08554 # in 1000 sq.km (1000 sq.km = 100 kha)
land_area_cell = 0.0779868 # for Indonesia in 1000 sq.km 
#land_area_cell = 0.079871871 # for Nicaragua and Honduras in 1000 sq.km
#land_area_cell = 0.08333 # for Nicaragua and Honduras in 1000 sq.km

input_data_location_yld = os.path.join(root.folder, 'indonesia_attainable_yield_all_crops.csv')
input_data_location_lnd = os.path.join(root.folder, 'indonesia_land_cover_types.csv')
input_data_location_reg = os.path.join(root.folder, 'indonesia_regions_v2.csv') # Location of file with regional division of GAEZ data

### Water clustering
water_included = True

if water_included:
    input_data_location_cwd = os.path.join(root.folder, 'indonesia_crop_water_deficit_all_crops.csv') # Location of file crop water deficit data
    input_data_location_evt = os.path.join(root.folder, 'indonesia_evapotranspiration_all_crops.csv') # Location of file evapotranspiration data
    input_data_location_prc = os.path.join(root.folder, 'indonesia_precipitation.csv') # Location of file precipitation data

write_to_csv = True # Set as 'True' or 'False' to write an output file or not  

plot_elbow = False # Set as 'True' or 'False' to plot 'Elbow graph' or not. The 'Elbow graph' helps select an appropriate number of clusters for a given dataset   
plot_clusters = False # Set as 'True' or 'False' to plot results by cluster or not.
plot_map = True # Set as 'True' or 'False' to plot map of cells by cluster or not.

In [ ]:
df = pd.read_csv(input_data_location_yld, header=None) # Read data for crop yields
df_lc = pd.read_csv(input_data_location_lnd, header=None) # Read data for land area by land cover type
df_regions = pd.read_csv(input_data_location_reg, header=None) # Read data file for regions

if water_included: 
    df_cwd = pd.read_csv(input_data_location_cwd, header=None) # Read data for crop water deficit
    df_evt = pd.read_csv(input_data_location_evt, header=None) # Read data for evapotranspiration
    df_prc = pd.read_csv(input_data_location_prc, header=None) # Read data for precipitation

def read_prep_df(df_name):
    column_names = ['column id', 'row number'] + list(range(0,len(df_name.columns)-2))
    df_name.columns = column_names
    df_name = df_name.melt(['column id','row number'], var_name='col number', value_name='region' if df_name['column id'].any() == 'region' else 'value') # Reformat table from wide to long
    df_name = df_name.pivot_table(index=['row number','col number'], columns='column id', values='region' if df_name['column id'].any() == 'region' else 'value') #Set crop combinations as columns
    return df_name

In [ ]:
# OPTIONAL: Plot values for each cluster

def print_clusters(df, Z, k, plot=False):
    # k Number of clusters I'd like to extract
    results = fcluster(Z, k, criterion='maxclust')

    # check the results
    s = pd.Series(results)
    clusters = s.unique()
    x = df_plot_2.columns
    x_ticks = pd.Series(x)    
    
    for c in clusters:
        cluster_indices = s[s==c].index
        #print("Cluster %d contains %d clusters" % (c, len(cluster_indices)))
        #print(cluster_indices)
        if plot:
            ax = df_plot_2.T.iloc[:,cluster_indices].plot(legend=None)
            ax.set_ylabel('Crop yield (t/ha)')
            ax.set_xlabel('Crop name')
            plt.xticks(np.arange(len(x)), x, rotation='vertical')
            plt.show()
            
def crop_selection(df_all):
    for each in df_all.columns:
        if ' '.join(each.split(' ')[:-2]) not in list(itertools.chain.from_iterable(crop_list.values())):
            df_all.drop(each, axis=1, inplace=True)
    return df_all

def filter_df(df_all):
    df_new = df_all.loc[df_all['region'] == region].drop('region', axis=1)
    df_new = df_new.replace(-9, np.nan)
    df_new = df_new.dropna(thresh=1)
    df_new[df_new<0]=0
    return df_new

In [ ]:
# Join region dataframe to crop yields and land cover data frames Region column
df = read_prep_df(df)
crop_selection(df)            

combine_list = {}
for each_crop in crop_list.keys():
        for each_column in df.columns:
            if ' '.join(each_column.split(' ')[:-2]) in crop_list[each_crop]:
                merge_column_name = str(each_crop) + ' ' + ' '.join(each_column.split(' ')[-2:])
                combine_list[merge_column_name] = [(str(sub_crop) + ' ' + ' '.join(each_column.split(' ')[-2:])) for sub_crop in crop_list[each_crop]]

df_lc = read_prep_df(df_lc)
df_lc = df_lc.mul(land_area_cell/100) # Calculate area of different land cover type for each cell in 1000 sq.km
df_regions = read_prep_df(df_regions)
df_regions = df_regions.astype(int)

    
df = df.join(df_regions)
df_lc = df_lc.join(df_regions)

if water_included:
    df_cwd = read_prep_df(df_cwd)
    crop_selection(df_cwd)
    df_cwd = df_cwd.join(df_regions)
    
    df_evt = read_prep_df(df_evt)
    crop_selection(df_evt) 
    df_evt = df_evt.join(df_regions)
    
    df_prc = read_prep_df(df_prc)
    df_prc = df_prc.join(df_regions)


# Filter dataframe to keep only selected region
for region in regions.keys():
    
    df_selected = filter_df(df)
    #df_selected = df.loc[df['region'] == region].drop('region', axis=1)
    df_lc_selected = df_lc.loc[df_lc['region'] == region].drop('region', axis=1)
    
    if water_included:
        #df_cwd_selected = df_cwd.loc[df_cwd['region'] == region].drop('region', axis=1)
        #df_evt_selected = df_evt.loc[df_evt['region'] == region].drop('region', axis=1)
        #df_prc_selected = df_prc.loc[df_prc['region'] == region].drop('region', axis=1)
        df_cwd_selected = filter_df(df_cwd)
        df_evt_selected = filter_df(df_evt)
        df_prc_selected = filter_df(df_prc)
        
    for each in combine_list:
        if each not in df_selected.columns:
            df_selected[each] = df_selected[combine_list[each]].mean(axis=1)
            df_selected.drop(combine_list[each], axis=1, inplace=True)
            if water_included:
                df_cwd_selected[each] = df_cwd_selected[combine_list[each]].mean(axis=1)
                df_cwd_selected.drop(combine_list[each], axis=1, inplace=True)
                df_evt_selected[each] = df_evt_selected[combine_list[each]].mean(axis=1)
                df_evt_selected.drop(combine_list[each], axis=1, inplace=True)

    df_selected = df_selected.reindex(sorted(df_selected.columns), axis=1)
    crop_order = list(df_selected.columns)
    
    # Data normalization using min-max (0-1)
    normalized_df = (df_selected-df_selected.min())/(df_selected.max()-df_selected.min())
    normalized_df.fillna(0, inplace=True)
    
    # Agglomerative Heirarchical clustering
    Z = hac.linkage(normalized_df, method='ward', metric='euclidean') # Select type of Agglomerative Heirarchical clustering
    results = fcluster(Z, k[region], criterion='maxclust') # Select cut-off. k = number of clusters
    s = pd.Series(results)
    
    # Save df_5 to plot clusters
    df_plot_2 = normalized_df
    
    # Plot map of cells by cluster
    df_selected['cluster'] = s.values
    
    if plot_map == True:
        plt.figure()
        df_plot_1 = df_selected[['cluster']]
        cluster_df = pd.DataFrame(df_plot_1['cluster'].value_counts()).sort_index().reset_index()
        cluster_df['label names'] = cluster_df['index'].astype(str) + ' (' + cluster_df['cluster'].astype(str) + ' cells)'
        labels = pd.Series(cluster_df['index'])
        label_names = pd.Series(cluster_df['label names'])
        
        cluster_map = df_plot_1.reset_index().pivot("row number", "col number", "cluster")
        sns.set_style('white')
        ax = sns.heatmap(cluster_map, cmap='viridis')
        ax.set_aspect('equal')
        
        colorbar = ax.collections[0].colorbar
        colorbar.set_ticks(labels)
        colorbar.set_ticklabels(label_names)
        
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title('Map of clusters for region ' + str(regions[region]))
        plt.show()
        
    print_clusters(df_selected, Z, k[region], plot=plot_clusters)
    
    # Plot 'Elbow' graph
    last = Z[-25:, 2]
    last_rev = last[::-1]
    error_ratio = (last_rev/max(last_rev)).round(2)
    idxs = np.arange(1, len(last) + 1)
    
    if plot_elbow==True:
        plt.figure()
        plt.plot(idxs, error_ratio, marker = 'o', markersize = 5)
        plt.xlabel('Number of clusters')
        plt.ylabel('% of max. error')
        plt.title('Elbow graph for region ' + str(regions[region]))
        
        for i,j in zip(idxs, error_ratio):
            plt.annotate(str(j), xy=(i,j), ha='left', fontsize = 12)
        plt.show()
    
    # Aggregate data into clusters. Sum the number of cells in each cluster and calculate mean scores for each crop combination
    for each in df_selected.columns:
        if each != 'cluster':
            df_selected[each] = df_selected[each].div(10)    
    df_selected = df_selected.join(df_lc_selected)
    df_selected['count'] = 1
    aggregator = {}
    
    for each in df_selected.columns:
        if each not in ['cluster','count'] + list(df_lc.columns) :
            aggregator[each] = 'mean'
        if each in list(df_lc.columns) :
            aggregator[each] = 'sum'
    aggregator['count'] = 'sum'
    
    df_6 = df_selected.groupby(['cluster']).agg(aggregator).astype(float) # Sum the number of cells in each cluster and calculate mean scores for each crop combination
    df_6['land_area_total'] = df_6['count']*land_area_cell # Calculate land area of each cluster
    #df_7 = df_6.drop(['count'], axis=1)
    
    # Reorder columns
    col_order = ['count','land_area_total'] + list(df_lc_selected.columns) + crop_order
    df_6 = df_6[col_order]
    
    if water_included:
        df_cluster = pd.DataFrame(df_selected['cluster'])
        
        df_cwd_selected = df_cluster.join(df_cwd_selected)
        df_cwd_selected = df_cwd_selected.groupby(['cluster']).agg('mean').astype(float).div(1000)
        
        df_evt_selected = df_cluster.join(df_evt_selected)
        df_evt_selected = df_evt_selected.groupby(['cluster']).agg('mean').astype(float).div(1000)
        
        df_prc_selected = df_cluster.join(df_prc_selected)
        df_prc_selected = df_prc_selected.groupby(['cluster']).agg('mean').astype(float).div(1000)

        
    # Print results in csv   
    if write_to_csv == True:
        df_6.to_csv(os.path.join(root.folder, 'clustering_results_' + str(regions[region]) +'.csv'))
        
        if water_included:
            df_cwd_selected.to_csv(os.path.join(root.folder, 'clustering_results_cwd_' + str(regions[region]) +'.csv'))
            df_evt_selected.to_csv(os.path.join(root.folder, 'clustering_results_evt_' + str(regions[region]) +'.csv'))
            df_prc_selected.to_csv(os.path.join(root.folder, 'clustering_results_prc_' + str(regions[region]) +'.csv'))
     